In [13]:
from bs4 import BeautifulSoup
import requests
import os
import numpy as np
import json

In [14]:
def get_company_names():                           # returns list of all companies
    with open('list_of_companies.json') as f:
        data=json.load(f)
    return data    

In [15]:
def remove(list,tag,replacement):           #replace the tag with replacement ---for removing the tag
    new_list=[]
    for review in list:
        temp=str(review).replace(tag,replacement)
        new_list.append(temp)
    return new_list  

In [16]:
#remove tags from the input data
#tags=['<span class="cmp-review-text" itemprop="reviewBody">','</span>','<br/>','<br>']
def remove_tags(data,tags):
    for tag in tags:
        data=remove(data,tag,' ')
    return data    


In [17]:
def get_count(page):                                  #returns total number of pages
    t=page.find_all('div',attrs={'class':'cmp-note'})
    count=remove(t[1],'<div class="cmp-note">',' ')
    count=remove(count,',','')
    count=(int(count[0])/20)+1
    return count

In [18]:
def fetch_page(url):                              #fetches current page with given url
    page=requests.get(url).text
    page=BeautifulSoup(page,'lxml')
    return page

In [25]:
def get_pages(cmpy_name):                        #get total no. of pages and traverse  through each page
    count=0
    url='https://www.indeed.co.in/cmp/'+cmpy_name+'/reviews?fcountry=ALL&start='+str(count)
    page=fetch_page(url)
    f=open('../Reviews/'+cmpy_name+'/review.txt','a+')
    total_page=get_count(page)
    for i in range(0,total_page):
        count=count*20
        url='https://www.indeed.co.in/cmp/'+cmpy_name+'/reviews?fcountry=ALL&start='+str(count)
        page=fetch_page(url)
        reviews=page.find_all('span',attrs={'class':'cmp-review-text'})
        tags=['<span class="cmp-review-text" itemprop="reviewBody">','</span>','<br/>','<br>']
        reviews=remove_tags(reviews,tags)
        f.write(str(reviews))
        print "one more page completed"
    f.close()    

In [20]:
def get_company_review(cmpy_name):                  # all reviews of a company
    location='../Reviews/'+cmpy_name
    if not os.path.exists(location):
        os.makedirs(location)
    location+='/review.txt'    
    f=os.open(location,os.O_CREAT)
    get_pages(cmpy_name)

In [36]:
def get_all_reviews():                               #store all company reviews  in a file
    list=get_company_names()
    list=list[6:7]                #  name of companies whose reviews need to be scrapped.
    for l in list:
        get_company_review(l['name'])
        print l['name']+'scrapped successfully'

In [ ]:
get_all_reviews()